# Filter bam files on quality 20

In [ ]:
q20=/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/q20
bams=$(ls /workspace/hramzr/2_Phd_PROJECT/WGS/IR/*.bam)
module load samtools/1.7
for bam in $bams
do
prefix=$(basename $bam | awk -F "_" '{print $1}')
echo "samtools view -hb -q 20 $bam -o ${q20}/${prefix}_q20.bam"
done | asub -j qfilter

# Index q20 bamfiles

In [ ]:
module load samtools/1.7
for bam in $(ls /workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/q20/*.bam)
do
    echo "samtools index $bam"
done | asub -j index

# Get Parliament2 ensemble caller Docker with singularity for usage on HPC server

In [ ]:
# download docker for singularity
module load singularity/2.6.1
# singularity pull docker://sameerdcosta/parliament2
bsub \
-J "pull singularity docker" \
"singularity pull --name parliament2.simg docker://dnanexus/parliament2:latest"

# Make separate directionaries for parliament

In [ ]:
bams=$(ls /workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/q20/*.bam)
calldir=/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/parliament2_calling
for bam in $bams
do
prefix=$(basename $bam | awk -F "_" '{print $1}')
mkdir ${calldir}/$prefix
mkdir ${calldir}/${prefix}/outputs
done

# Load bam, index and reference files into parliament directionaries

In [ ]:
bams=$(ls /workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/q20/*967038331*.bam)
calldir=/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/parliament2_calling/
ref=/workspace/hramzr/2_Phd_PROJECT/snapper_genome/Chrysophrys_auratus.v.1.0.chromosomes.male.map.fasta 
fai=${ref}.fai
log=/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/logs
for bam in $bams
do
prefix=$(basename $bam | awk -F "_" '{print $1}')
bai=$(ls /workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/q20/${prefix}*.bai)
# echo $bai
# echo $bam
# echo $prefix
# echo $fai
# echo $ref
bsub -o ${log}/cp.o -e ${log}/cp.e \
"cp $bam ${calldir}${prefix}/"
bsub -o ${log}/cp.o -e ${log}/cp.e \
"cp $bai ${calldir}${prefix}/"
bsub -o ${log}/cp.o -e ${log}/cp.e \
"cp $ref ${calldir}${prefix}/"
bsub -o ${log}/cp.o -e ${log}/cp.e \
"cp $fai ${calldir}${prefix}/"
bsub -o ${log}/cp.o -e ${log}/cp.e \
"cp /workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/parliament2.simg ${calldir}${prefix}/"
done

# Run parliament2

In [ ]:
module load singularity/2.6.1
bams=$(ls /workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/q20/*967038331*.bam)
wdir=/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/parliament2_calling/
out=/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/parliament2_out
indir=/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/
ref=Chrysophrys_auratus.v.1.0.chromosomes.male.map.fasta
fai=Chrysophrys_auratus.v.1.0.chromosomes.male.map.fasta.fai
log=/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/logs/
for bam in $bams
do
prefix=$(basename $bam | awk -F "_" '{print $1}')
cd ${wdir}${prefix}
bsub \
-o ${log}sing.out -e ${log}sing.err -J "sing" \
-R "rusage[mem=64000] span[hosts=1]" \
"singularity run -B ${wdir}${prefix}:/home/dnanexus/in,${wdir}${prefix}/outputs:/home/dnanexus/out parliament2.simg \
--bam ${prefix}_q20.bam --bai ${prefix}_q20.bam.bai --fai $fai -r $ref \
    --filter_short_contigs \
    --breakdancer \
    --breakseq \
    --manta \
    --cnvnator \
    --lumpy \
    --delly_deletion \
    --delly_insertion \
    --delly_inversion \
    --delly_duplication \
    --genotype"
done 

# Run FreeBayes

In [ ]:
module load freebayes/1.1.0
module load tabix/0.2.6
module load samtools/1.7
export TMPDIR=/workspace/hramzr/2_Phd_PROJECT/GBS/vcf_snapper/tmp

OUT="/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/freebayes"
LOG="${OUT}/logs"
SAMPLES="/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/help_files/indvs"
PAR="/software/bioinformatics/freebayes-1.1.0/scripts"
ref=/workspace/hramzr/2_Phd_PROJECT/snapper_genome/Chrysophrys_auratus.v.1.0.chromosomes.male.map.fasta 
fai=${ref}.fai
TEMP=/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/freebayes/tmp

bsub << EOF
#!/bin/bash
#BSUB -J "Free"
#BSUB -o ${LOG}/freeb.out 
#BSUB -e ${LOG}/freeb.err
#BSUB -n 80
#BSUB -R "span[hosts=1]"

freebayes-parallel <($PAR/fasta_generate_regions.py $fai 1000000) 80 \
             -f ${ref} \
             -L ${SAMPLES} \
             --ploidy 2 \
             --report-genotype-likelihood-max \
             --min-base-quality 10 \
             --min-mapping-quality 20 \
             --genotype-qualities \
             --use-mapping-quality \
             --no-mnps \
             --no-complex \
             --max-complex-gap 50 \
             --min-alternate-fraction 0.1 \
             --min-repeat-entropy 1 \
             --no-partial-observations \
             --min-coverage 10 \
             --max-coverage 500 \
             --pooled-continuous>${OUT}/80_Samples_FB.vcf
EOF